# Testing dots.ocr for Bengali OCR

This notebook tests the [rednote-hilab/dots.ocr](https://huggingface.co/rednote-hilab/dots.ocr) model for Bengali language OCR capabilities.

dots.ocr is a powerful 3.04B parameter multilingual document parser that unifies layout detection and content recognition within a single vision-language model.


In [1]:
# Install required dependencies
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install transformers>=4.37.0
%pip install qwen-vl-utils
%pip install accelerate
%pip install pillow
%pip install requests
%pip install matplotlib
%pip install numpy
%pip install opencv-python

# Try to install flash-attention (may fail on some systems)
try:
    import subprocess
    import sys
    result = subprocess.run([sys.executable, "-m", "pip", "install", "flash-attn", "--no-build-isolation"], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print("Flash attention installed successfully")
    else:
        print("Flash attention installation failed, continuing without it...")
except Exception as e:
    print(f"Flash attention installation error: {e}")

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Flash attention installed successfully


In [2]:
# Import required libraries
import torch
from transformers import AutoModelForCausalLM, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import matplotlib.pyplot as plt
import json
import requests
from io import BytesIO
import os
from huggingface_hub import login
import gc

2025-08-06 11:00:18.427908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754478018.451585     224 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754478018.458708     224 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 3. Model Loading and Setup

In [4]:
# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Using device: cuda
GPU: Tesla T4
CUDA Memory: 14.7 GB


In [5]:
# Load the model and processor
model_id = "rednote-hilab/dots.ocr"

print("Loading processor...")
processor = AutoProcessor.from_pretrained(
    model_id, 
    trust_remote_code=True,
    token=HF_TOKEN  # Updated parameter name
)

print("Loading model...")
# Load model with optimizations for Kaggle environment
try:
    # Try with flash attention first
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,  # Use bfloat16 for memory efficiency
        device_map="auto",  # Automatically distribute across available devices
        trust_remote_code=True,
        token=HF_TOKEN,  # Updated parameter name
        low_cpu_mem_usage=True,  # Reduce CPU memory usage during loading
        attn_implementation="flash_attention_2"  # Use flash attention for efficiency
    )
    print("Model loaded with flash attention!")
except Exception as e:
    print(f"Flash attention failed: {e}")
    print("Loading model without flash attention...")
    # Fallback without flash attention
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        token=HF_TOKEN,
        low_cpu_mem_usage=True
    )
    print("Model loaded without flash attention!")

print(f"Model device: {model.device}")
print(f"Model dtype: {model.dtype}")


Loading processor...
Loading model...
Flash attention failed: No module named 'transformers_modules.rednote-hilab.dots'
Loading model without flash attention...


ModuleNotFoundError: No module named 'transformers_modules.rednote-hilab.dots'

In [6]:
import os
import sys
import shutil
from pathlib import Path

def fix_dots_ocr_modules():
    """
    Fix the transformers_modules issue for dots.ocr
    """
    print("Attempting to fix dots.ocr module loading...")
    
    # Get the transformers cache directory
    from transformers.utils import TRANSFORMERS_CACHE
    cache_dir = Path(TRANSFORMERS_CACHE)
    
    # Look for the dots.ocr modules
    modules_dir = cache_dir / "modules" / "transformers_modules" / "rednote-hilab" / "dots.ocr"
    
    if modules_dir.exists():
        print(f"Found dots.ocr modules at: {modules_dir}")
        
        # List all subdirectories (different versions/commits)
        for version_dir in modules_dir.iterdir():
            if version_dir.is_dir():
                print(f"Found version: {version_dir.name}")
                
                # Add the directory to Python path
                if str(version_dir) not in sys.path:
                    sys.path.insert(0, str(version_dir))
                    print(f"Added {version_dir} to Python path")
                
                # Try to create the missing module structure
                try:
                    init_file = version_dir / "__init__.py"
                    if not init_file.exists():
                        init_file.touch()
                        print(f"Created __init__.py in {version_dir}")
                except Exception as e:
                    print(f"Could not create __init__.py: {e}")
    else:
        print("dots.ocr modules not found in cache")
    
    return modules_dir.exists()

# Try to fix the module issue
if fix_dots_ocr_modules():
    print("Module fix attempted. Trying to load model...")
else:
    print("Could not find cached modules. Model may need to be downloaded first.")

# Clear any problematic cached imports
modules_to_clear = [k for k in sys.modules.keys() if 'transformers_modules' in k or 'dots' in k]
for module in modules_to_clear:
    del sys.modules[module]
    
print(f"Cleared {len(modules_to_clear)} cached modules")


Attempting to fix dots.ocr module loading...
dots.ocr modules not found in cache
Could not find cached modules. Model may need to be downloaded first.
Cleared 4 cached modules


In [7]:
# COMPLETE FIX for dots.ocr module loading issue
import sys
import os
from pathlib import Path
from transformers.utils import TRANSFORMERS_CACHE

def fix_dots_ocr_imports():
    """
    Fix the broken relative imports in dots.ocr custom modules
    """
    print("Fixing dots.ocr module imports...")
    
    # Find the dots.ocr module directory
    cache_dir = Path(TRANSFORMERS_CACHE)
    dots_module_dir = cache_dir / "modules" / "transformers_modules" / "rednote-hilab" / "dots.ocr"
    
    if not dots_module_dir.exists():
        print("dots.ocr module directory not found")
        return False
    
    # Find the specific version directory
    version_dirs = [d for d in dots_module_dir.iterdir() if d.is_dir()]
    if not version_dirs:
        print("No version directories found")
        return False
    
    version_dir = version_dirs[0]  # Use the first (and likely only) version
    print(f"Found version directory: {version_dir}")
    
    # Add the version directory to Python path
    if str(version_dir) not in sys.path:
        sys.path.insert(0, str(version_dir))
        print(f"Added {version_dir} to Python path")
    
    # Create proper __init__.py files
    init_files = [
        version_dir / "__init__.py",
        version_dir.parent / "__init__.py",
        version_dir.parent.parent / "__init__.py",
        version_dir.parent.parent.parent / "__init__.py"
    ]
    
    for init_file in init_files:
        if not init_file.exists():
            try:
                init_file.parent.mkdir(parents=True, exist_ok=True)
                init_file.write_text("")
                print(f"Created {init_file}")
            except Exception as e:
                print(f"Could not create {init_file}: {e}")
    
    # Fix the broken import in modeling_dots_ocr.py
    modeling_file = version_dir / "modeling_dots_ocr.py"
    if modeling_file.exists():
        try:
            content = modeling_file.read_text()
            
            # Replace the problematic relative import
            old_import = "from .modeling_dots_vision import DotsVisionTransformer"
            new_import = """
try:
    from .modeling_dots_vision import DotsVisionTransformer
except ImportError:
    # Fallback for broken relative imports
    import importlib.util
    import sys
    
    vision_file = Path(__file__).parent / "modeling_dots_vision.py"
    spec = importlib.util.spec_from_file_location("modeling_dots_vision", vision_file)
    vision_module = importlib.util.module_from_spec(spec)
    sys.modules["modeling_dots_vision"] = vision_module
    spec.loader.exec_module(vision_module)
    DotsVisionTransformer = vision_module.DotsVisionTransformer
"""
            
            if old_import in content:
                content = content.replace(old_import, new_import)
                modeling_file.write_text(content)
                print("Fixed import in modeling_dots_ocr.py")
            else:
                print("Import already fixed or not found")
                
        except Exception as e:
            print(f"Could not fix modeling file: {e}")
    
    return True

# Apply the fix
if fix_dots_ocr_imports():
    print("Import fix applied successfully!")
    
    # Clear any cached modules that might have the old broken imports
    modules_to_clear = [k for k in list(sys.modules.keys()) if 'dots' in k.lower() or 'transformers_modules' in k]
    for module in modules_to_clear:
        if module in sys.modules:
            del sys.modules[module]
    
    print(f"Cleared {len(modules_to_clear)} cached modules")
    
    # Now try to load the model
    try:
        print("Attempting to load model with fixed imports...")
        
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True,
            token=HF_TOKEN,
            low_cpu_mem_usage=True
        )
        
        print("✅ Model loaded successfully!")
        print(f"Model device: {model.device}")
        print(f"Model dtype: {model.dtype}")
        
    except Exception as e:
        print(f"❌ Model loading still failed: {e}")
        print("\nTrying alternative approach...")
        
        # Alternative: Force download and try again
        try:
            from huggingface_hub import snapshot_download
            snapshot_download(
                repo_id=model_id,
                force_download=True,
                token=HF_TOKEN
            )
            print("Re-downloaded model files")
            
            # Try loading again
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True,
                token=HF_TOKEN,
                force_download=True
            )
            print("✅ Model loaded after re-download!")
            
        except Exception as e2:
            print(f"❌ All methods failed: {e2}")
            print("Please restart kernel and try again")
            
else:
    print("Could not apply import fix")

Fixing dots.ocr module imports...
dots.ocr module directory not found
Could not apply import fix


In [8]:
# COMPLETE SOLUTION: Download first, then fix imports
import sys
import os
from pathlib import Path
from transformers.utils import TRANSFORMERS_CACHE
from huggingface_hub import snapshot_download

def download_and_fix_dots_ocr():
    """
    Download dots.ocr model files and fix the import issues
    """
    print("Step 1: Downloading dots.ocr model files...")
    
    try:
        # Force download all model files
        local_dir = snapshot_download(
            repo_id="rednote-hilab/dots.ocr",
            token=HF_TOKEN,
            force_download=True,
            resume_download=False
        )
        print(f"✅ Downloaded model files to: {local_dir}")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        return False
    
    print("\nStep 2: Locating and fixing module imports...")
    
    # Now find the downloaded modules
    cache_dir = Path(TRANSFORMERS_CACHE)
    dots_module_dir = cache_dir / "modules" / "transformers_modules" / "rednote-hilab" / "dots.ocr"
    
    if not dots_module_dir.exists():
        print("Still no dots.ocr module directory found after download")
        return False
    
    # Find version directory
    version_dirs = [d for d in dots_module_dir.iterdir() if d.is_dir()]
    if not version_dirs:
        print("No version directories found")
        return False
    
    version_dir = version_dirs[0]
    print(f"Found version directory: {version_dir}")
    
    # Add to Python path
    if str(version_dir) not in sys.path:
        sys.path.insert(0, str(version_dir))
        print(f"Added to Python path: {version_dir}")
    
    # Create __init__.py files
    init_paths = [
        version_dir / "__init__.py",
        version_dir.parent / "__init__.py", 
        version_dir.parent.parent / "__init__.py",
        cache_dir / "modules" / "__init__.py",
        cache_dir / "modules" / "transformers_modules" / "__init__.py",
        cache_dir / "modules" / "transformers_modules" / "rednote-hilab" / "__init__.py"
    ]
    
    for init_path in init_paths:
        try:
            init_path.parent.mkdir(parents=True, exist_ok=True)
            if not init_path.exists():
                init_path.write_text("")
                print(f"Created: {init_path}")
        except Exception as e:
            print(f"Could not create {init_path}: {e}")
    
    # Fix the problematic import in modeling_dots_ocr.py
    modeling_file = version_dir / "modeling_dots_ocr.py"
    if modeling_file.exists():
        print("Fixing modeling_dots_ocr.py imports...")
        try:
            content = modeling_file.read_text()
            
            # Replace the broken import line
            old_line = "from .modeling_dots_vision import DotsVisionTransformer"
            
            new_lines = '''# Fixed import for DotsVisionTransformer
import sys
from pathlib import Path
try:
    from .modeling_dots_vision import DotsVisionTransformer
except ImportError:
    # Manual import fallback
    current_dir = Path(__file__).parent
    vision_file = current_dir / "modeling_dots_vision.py"
    if vision_file.exists():
        import importlib.util
        spec = importlib.util.spec_from_file_location("modeling_dots_vision", vision_file)
        vision_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(vision_module)
        DotsVisionTransformer = vision_module.DotsVisionTransformer
    else:
        raise ImportError("Could not find modeling_dots_vision.py")'''
            
            if old_line in content:
                content = content.replace(old_line, new_lines)
                modeling_file.write_text(content)
                print("✅ Fixed import in modeling_dots_ocr.py")
            else:
                print("Import line not found or already fixed")
                
        except Exception as e:
            print(f"❌ Could not fix modeling file: {e}")
            return False
    else:
        print("❌ modeling_dots_ocr.py not found")
        return False
    
    return True

# Execute the fix
print("Starting complete dots.ocr fix process...")
print("=" * 50)

if download_and_fix_dots_ocr():
    print("\n" + "=" * 50)
    print("✅ Download and fix completed successfully!")
    
    # Clear any cached broken modules
    modules_to_clear = [k for k in list(sys.modules.keys()) 
                       if any(x in k.lower() for x in ['dots', 'transformers_modules'])]
    for module in modules_to_clear:
        if module in sys.modules:
            del sys.modules[module]
    print(f"Cleared {len(modules_to_clear)} cached modules")
    
    print("\nStep 3: Attempting to load the model...")
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto", 
            trust_remote_code=True,
            token=HF_TOKEN,
            low_cpu_mem_usage=True,
            local_files_only=False  # Allow using downloaded files
        )
        
        print("🎉 SUCCESS! Model loaded successfully!")
        print(f"Model device: {model.device}")
        print(f"Model dtype: {model.dtype}")
        
    except Exception as e:
        print(f"❌ Model loading failed even after fix: {e}")
        
        # Last resort: try with different parameters
        print("\nTrying with alternative loading parameters...")
        try:
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.float16,  # Different dtype
                device_map="cpu",  # Force CPU first
                trust_remote_code=True,
                token=HF_TOKEN
            )
            print("✅ Model loaded on CPU!")
            
            # Move to GPU if available
            if torch.cuda.is_available():
                model = model.to("cuda")
                print("✅ Moved model to GPU!")
                
        except Exception as e2:
            print(f"❌ All loading attempts failed: {e2}")
            print("\n🔄 Please restart the kernel and try again.")
            
else:
    print("❌ Download and fix process failed")
    print("Please check your HF_TOKEN and internet connection")

Starting complete dots.ocr fix process...
Step 1: Downloading dots.ocr model files...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

configuration_dots.py: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

✅ Downloaded model files to: /root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f

Step 2: Locating and fixing module imports...
Still no dots.ocr module directory found after download
❌ Download and fix process failed
Please check your HF_TOKEN and internet connection


In [9]:
# CORRECT APPROACH: Trigger module creation then fix
import sys
import os
from pathlib import Path
from transformers.utils import TRANSFORMERS_CACHE

def trigger_and_fix_dots_ocr():
    """
    Trigger the module creation by attempting to load, then fix the imports
    """
    print("Step 1: Triggering module creation...")
    
    # First, try to load the model to trigger module creation
    # This will fail but will create the modules directory
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True,
            token=HF_TOKEN
        )
        print("Model loaded successfully on first try!")
        return True, model
    except Exception as e:
        print(f"Expected failure (this creates the modules): {e}")
    
    print("\nStep 2: Looking for created modules...")
    
    # Now the modules should exist
    cache_dir = Path(TRANSFORMERS_CACHE)
    dots_module_dir = cache_dir / "modules" / "transformers_modules" / "rednote-hilab" / "dots.ocr"
    
    if not dots_module_dir.exists():
        print(f"Module directory still not found at: {dots_module_dir}")
        print("Let's check what was created...")
        
        # Debug: show what's in the cache
        modules_dir = cache_dir / "modules"
        if modules_dir.exists():
            print(f"Contents of {modules_dir}:")
            for item in modules_dir.rglob("*"):
                if "dots" in str(item).lower():
                    print(f"  Found: {item}")
        
        return False, None
    
    # Find the version directory
    version_dirs = [d for d in dots_module_dir.iterdir() if d.is_dir()]
    if not version_dirs:
        print("No version directories found")
        return False, None
    
    version_dir = version_dirs[0]
    print(f"Found version directory: {version_dir}")
    
    print("\nStep 3: Fixing the import issue...")
    
    # Add to Python path
    if str(version_dir) not in sys.path:
        sys.path.insert(0, str(version_dir))
        print(f"Added to Python path: {version_dir}")
    
    # Create __init__.py files for proper module structure
    init_files = [
        cache_dir / "modules" / "__init__.py",
        cache_dir / "modules" / "transformers_modules" / "__init__.py", 
        cache_dir / "modules" / "transformers_modules" / "rednote-hilab" / "__init__.py",
        dots_module_dir / "__init__.py",
        version_dir / "__init__.py"
    ]
    
    for init_file in init_files:
        try:
            init_file.parent.mkdir(parents=True, exist_ok=True)
            if not init_file.exists():
                init_file.write_text("")
                print(f"Created: {init_file.name}")
        except Exception as e:
            print(f"Could not create {init_file}: {e}")
    
    # Fix the broken import in modeling_dots_ocr.py
    modeling_file = version_dir / "modeling_dots_ocr.py"
    
    if not modeling_file.exists():
        print(f"❌ modeling_dots_ocr.py not found at {modeling_file}")
        return False, None
    
    print("Patching modeling_dots_ocr.py...")
    
    try:
        content = modeling_file.read_text()
        
        # Check if already patched
        if "# PATCHED IMPORT" in content:
            print("File already patched")
        else:
            # Replace the problematic import
            old_import = "from .modeling_dots_vision import DotsVisionTransformer"
            
            patched_import = """# PATCHED IMPORT - Fixed relative import issue
import sys
from pathlib import Path

try:
    from .modeling_dots_vision import DotsVisionTransformer
except (ImportError, ModuleNotFoundError):
    # Fallback: direct file import
    current_file = Path(__file__)
    vision_file = current_file.parent / "modeling_dots_vision.py"
    
    if vision_file.exists():
        import importlib.util
        spec = importlib.util.spec_from_file_location("modeling_dots_vision", vision_file)
        vision_module = importlib.util.module_from_spec(spec)
        sys.modules[f"{__name__}.modeling_dots_vision"] = vision_module
        spec.loader.exec_module(vision_module)
        DotsVisionTransformer = vision_module.DotsVisionTransformer
        print(f"Successfully imported DotsVisionTransformer via fallback")
    else:
        raise ImportError(f"Could not find modeling_dots_vision.py at {vision_file}")"""
            
            if old_import in content:
                content = content.replace(old_import, patched_import)
                modeling_file.write_text(content)
                print("✅ Successfully patched modeling_dots_ocr.py")
            else:
                print("⚠️ Import line not found - file may have different structure")
                print("First few lines of the file:")
                print("\n".join(content.split("\n")[:15]))
    
    except Exception as e:
        print(f"❌ Could not patch file: {e}")
        return False, None
    
    # Clear any cached broken imports
    modules_to_clear = [k for k in list(sys.modules.keys()) 
                       if any(x in k for x in ['dots', 'transformers_modules'])]
    for module in modules_to_clear:
        if module in sys.modules:
            del sys.modules[module]
    print(f"Cleared {len(modules_to_clear)} cached modules")
    
    print("\nStep 4: Attempting to load model with fixed imports...")
    
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True,
            token=HF_TOKEN,
            low_cpu_mem_usage=True
        )
        
        print("🎉 SUCCESS! Model loaded with patched imports!")
        return True, model
        
    except Exception as e:
        print(f"❌ Model loading still failed: {e}")
        
        # Try one more approach: force local files only
        try:
            print("Trying with local_files_only=True...")
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True,
                token=HF_TOKEN,
                local_files_only=True
            )
            print("✅ Model loaded with local files!")
            return True, model
        except Exception as e2:
            print(f"❌ Final attempt failed: {e2}")
            return False, None

# Execute the complete fix
print("=" * 60)
print("COMPLETE DOTS.OCR LOADING AND FIXING PROCESS")
print("=" * 60)

success, model = trigger_and_fix_dots_ocr()

if success:
    print(f"\n🎉 FINAL SUCCESS!")
    print(f"Model device: {model.device}")
    print(f"Model dtype: {model.dtype}")
    print(f"Model loaded and ready for Bengali OCR testing!")
else:
    print(f"\n❌ Process failed. Let's try a manual approach...")
    
    # Manual debug approach
    print("Checking transformers cache manually...")
    cache_dir = Path(TRANSFORMERS_CACHE)
    print(f"Cache directory: {cache_dir}")
    
    if cache_dir.exists():
        for item in cache_dir.rglob("*dots*"):
            print(f"Found dots-related: {item}")

COMPLETE DOTS.OCR LOADING AND FIXING PROCESS
Step 1: Triggering module creation...
Expected failure (this creates the modules): No module named 'transformers_modules.rednote-hilab.dots'

Step 2: Looking for created modules...
Module directory still not found at: /root/.cache/huggingface/hub/modules/transformers_modules/rednote-hilab/dots.ocr
Let's check what was created...

❌ Process failed. Let's try a manual approach...
Checking transformers cache manually...
Cache directory: /root/.cache/huggingface/hub
Found dots-related: /root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr
Found dots-related: /root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f/configuration_dots.py
Found dots-related: /root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f/modeling_dots_vision.py
Found dots-related: /root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f417

In [10]:
# DIRECT FIX: Work with the actual downloaded files
import sys
import shutil
from pathlib import Path
from transformers.utils import TRANSFORMERS_CACHE

def direct_fix_dots_ocr():
    """
    Directly fix the downloaded files and create proper module structure
    """
    print("Direct fix approach using actual downloaded files...")
    
    # Locate the actual downloaded files
    hub_cache = Path("/root/.cache/huggingface/hub")
    model_dir = hub_cache / "models--rednote-hilab--dots.ocr"
    
    # Find the snapshot directory
    snapshots_dir = model_dir / "snapshots"
    if not snapshots_dir.exists():
        print("❌ Snapshots directory not found")
        return False
    
    # Get the latest snapshot
    snapshot_dirs = [d for d in snapshots_dir.iterdir() if d.is_dir()]
    if not snapshot_dirs:
        print("❌ No snapshot directories found")
        return False
    
    snapshot_dir = snapshot_dirs[0]  # Use the first (latest) snapshot
    print(f"✅ Found snapshot: {snapshot_dir}")
    
    # Check for the problematic files
    modeling_ocr_file = snapshot_dir / "modeling_dots_ocr.py"
    modeling_vision_file = snapshot_dir / "modeling_dots_vision.py"
    
    if not modeling_ocr_file.exists():
        print("❌ modeling_dots_ocr.py not found")
        return False
    
    if not modeling_vision_file.exists():
        print("❌ modeling_dots_vision.py not found")  
        return False
    
    print("✅ Found both modeling files")
    
    # Add the snapshot directory to Python path
    if str(snapshot_dir) not in sys.path:
        sys.path.insert(0, str(snapshot_dir))
        print(f"✅ Added to Python path: {snapshot_dir}")
    
    # Create the transformers_modules directory structure that transformers expects
    transformers_cache = Path(TRANSFORMERS_CACHE)
    modules_dir = transformers_cache / "modules" / "transformers_modules" / "rednote-hilab" / "dots.ocr"
    
    # Create a version directory (use the snapshot hash)
    version_dir = modules_dir / snapshot_dir.name
    version_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"✅ Created modules directory: {version_dir}")
    
    # Copy the files to the expected location
    files_to_copy = [
        "modeling_dots_ocr.py",
        "modeling_dots_vision.py", 
        "configuration_dots.py"
    ]
    
    for file_name in files_to_copy:
        source = snapshot_dir / file_name
        dest = version_dir / file_name
        if source.exists():
            shutil.copy2(source, dest)
            print(f"✅ Copied {file_name}")
        else:
            print(f"⚠️ {file_name} not found")
    
    # Create __init__.py files
    init_files = [
        transformers_cache / "modules" / "__init__.py",
        transformers_cache / "modules" / "transformers_modules" / "__init__.py",
        transformers_cache / "modules" / "transformers_modules" / "rednote-hilab" / "__init__.py",
        modules_dir / "__init__.py",
        version_dir / "__init__.py"
    ]
    
    for init_file in init_files:
        if not init_file.exists():
            init_file.write_text("")
            print(f"✅ Created {init_file.name}")
    
    # Now fix the import issue in the copied file
    modeling_file = version_dir / "modeling_dots_ocr.py"
    
    print("🔧 Fixing import in modeling_dots_ocr.py...")
    
    try:
        content = modeling_file.read_text()
        
        # Find and replace the problematic import
        old_import = "from .modeling_dots_vision import DotsVisionTransformer"
        
        # Create a more robust import
        new_import = """# FIXED IMPORT
import sys
from pathlib import Path
import importlib.util

def import_dots_vision():
    # Try relative import first
    try:
        from .modeling_dots_vision import DotsVisionTransformer
        return DotsVisionTransformer
    except ImportError:
        pass
    
    # Try absolute import from current directory
    try:
        current_dir = Path(__file__).parent
        vision_file = current_dir / "modeling_dots_vision.py"
        
        if vision_file.exists():
            spec = importlib.util.spec_from_file_location("modeling_dots_vision", vision_file)
            vision_module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(vision_module)
            return vision_module.DotsVisionTransformer
    except Exception as e:
        print(f"Import fallback failed: {e}")
    
    # Last resort: try from sys.path
    try:
        import modeling_dots_vision
        return modeling_dots_vision.DotsVisionTransformer
    except ImportError:
        pass
    
    raise ImportError("Could not import DotsVisionTransformer from any method")

DotsVisionTransformer = import_dots_vision()"""
        
        if old_import in content:
            content = content.replace(old_import, new_import)
            modeling_file.write_text(content)
            print("✅ Successfully fixed the import!")
        else:
            print("⚠️ Import line not found, checking file structure...")
            lines = content.split('\n')[:10]
            for i, line in enumerate(lines):
                print(f"{i+1}: {line}")
    
    except Exception as e:
        print(f"❌ Could not fix import: {e}")
        return False
    
    # Clear any cached modules
    modules_to_clear = [k for k in list(sys.modules.keys()) 
                       if any(x in k for x in ['dots', 'transformers_modules'])]
    for module in modules_to_clear:
        if module in sys.modules:
            del sys.modules[module]
    
    print(f"✅ Cleared {len(modules_to_clear)} cached modules")
    
    return True

# Execute the direct fix
print("🚀 Starting direct fix process...")
print("=" * 50)

if direct_fix_dots_ocr():
    print("\n✅ Direct fix completed! Now attempting to load model...")
    
    try:
        # Clear any remaining cached imports
        import gc
        gc.collect()
        
        # Try loading the model
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True,
            token=HF_TOKEN,
            low_cpu_mem_usage=True
        )
        
        print("🎉 HUGE SUCCESS! Model loaded successfully!")
        print(f"Model device: {model.device}")
        print(f"Model dtype: {model.dtype}")
        print(f"Model is ready for Bengali OCR testing! 🎯")
        
    except Exception as e:
        print(f"❌ Model loading failed: {e}")
        
        # One more try with different parameters
        print("\n🔄 Trying alternative loading parameters...")
        try:
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                device_map="cpu",
                trust_remote_code=True,
                token=HF_TOKEN
            )
            print("✅ Model loaded on CPU!")
            
            # Move to GPU if available
            if torch.cuda.is_available():
                try:
                    model = model.to("cuda")
                    print("✅ Successfully moved to GPU!")
                except:
                    print("⚠️ Staying on CPU due to memory constraints")
                    
        except Exception as e2:
            print(f"❌ All attempts failed: {e2}")
            print("\n💡 Try restarting the kernel and running this fix again")
            
else:
    print("❌ Direct fix failed")

🚀 Starting direct fix process...
Direct fix approach using actual downloaded files...
✅ Found snapshot: /root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f
✅ Found both modeling files
✅ Added to Python path: /root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f
✅ Created modules directory: /root/.cache/huggingface/hub/modules/transformers_modules/rednote-hilab/dots.ocr/42f41787f2475c0894d75a9e30e49cc1a6df519f
✅ Copied modeling_dots_ocr.py
✅ Copied modeling_dots_vision.py
✅ Copied configuration_dots.py
✅ Created __init__.py
✅ Created __init__.py
✅ Created __init__.py
✅ Created __init__.py
✅ Created __init__.py
🔧 Fixing import in modeling_dots_ocr.py...
✅ Successfully fixed the import!
✅ Cleared 4 cached modules

✅ Direct fix completed! Now attempting to load model...
❌ Model loading failed: No module named 'transformers_modules.rednote-hilab.dots'

🔄 Trying alternative l

In [11]:
# NUCLEAR OPTION: Direct Python import system patching
import sys
import importlib.util
from pathlib import Path

def nuclear_fix_dots_ocr():
    """
    Directly patch Python's import system to handle the broken module structure
    """
    print("🚀 Nuclear option: Direct import system patching...")
    
    # Get the actual file paths
    snapshot_dir = Path("/root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f")
    
    modeling_ocr_file = snapshot_dir / "modeling_dots_ocr.py"
    modeling_vision_file = snapshot_dir / "modeling_dots_vision.py"
    config_file = snapshot_dir / "configuration_dots.py"
    
    if not all([modeling_ocr_file.exists(), modeling_vision_file.exists(), config_file.exists()]):
        print("❌ Required files not found")
        return False
    
    print("✅ All required files found")
    
    # STEP 1: Manually load and register the modules
    print("📦 Manually loading modules...")
    
    try:
        # Load configuration_dots module
        spec = importlib.util.spec_from_file_location("configuration_dots", config_file)
        config_module = importlib.util.module_from_spec(spec)
        sys.modules["configuration_dots"] = config_module
        sys.modules["transformers_modules.rednote-hilab.dots.configuration_dots"] = config_module
        spec.loader.exec_module(config_module)
        print("✅ Loaded configuration_dots")
        
        # Load modeling_dots_vision module  
        spec = importlib.util.spec_from_file_location("modeling_dots_vision", modeling_vision_file)
        vision_module = importlib.util.module_from_spec(spec)
        sys.modules["modeling_dots_vision"] = vision_module
        sys.modules["transformers_modules.rednote-hilab.dots.modeling_dots_vision"] = vision_module
        spec.loader.exec_module(vision_module)
        print("✅ Loaded modeling_dots_vision")
        
        # Create a patched version of modeling_dots_ocr that uses our loaded modules
        ocr_content = modeling_ocr_file.read_text()
        
        # Replace the problematic imports with direct references
        patched_content = ocr_content.replace(
            "from .configuration_dots import DotsVisionConfig, DotsOCRConfig",
            "DotsVisionConfig = sys.modules['configuration_dots'].DotsVisionConfig\nDotsOCRConfig = sys.modules['configuration_dots'].DotsOCRConfig"
        ).replace(
            "from .modeling_dots_vision import DotsVisionTransformer", 
            "DotsVisionTransformer = sys.modules['modeling_dots_vision'].DotsVisionTransformer"
        )
        
        # Add sys import at the top
        if "import sys" not in patched_content:
            patched_content = "import sys\n" + patched_content
        
        # Load the patched OCR module
        spec = importlib.util.spec_from_loader("modeling_dots_ocr", loader=None)
        ocr_module = importlib.util.module_from_spec(spec)
        
        # Execute the patched content in the module's namespace
        exec(patched_content, ocr_module.__dict__)
        
        # Register the module in all the places transformers might look
        sys.modules["modeling_dots_ocr"] = ocr_module
        sys.modules["transformers_modules.rednote-hilab.dots.modeling_dots_ocr"] = ocr_module
        sys.modules["transformers_modules.rednote-hilab.dots"] = ocr_module  # This is the key one!
        
        print("✅ Loaded and patched modeling_dots_ocr")
        
        # Create a fake package structure
        class FakeModule:
            def __init__(self, name):
                self.__name__ = name
                self.__path__ = []
        
        # Create the missing package hierarchy
        sys.modules["transformers_modules"] = FakeModule("transformers_modules")
        sys.modules["transformers_modules.rednote-hilab"] = FakeModule("transformers_modules.rednote-hilab")
        
        print("✅ Created fake package structure")
        
    except Exception as e:
        print(f"❌ Module loading failed: {e}")
        import traceback
        traceback.print_exc()
        return False
    
    return True

# Execute nuclear fix
print("💥 NUCLEAR OPTION ACTIVATED")
print("=" * 50)

if nuclear_fix_dots_ocr():
    print("\n✅ Nuclear fix completed! Testing model loading...")
    
    try:
        # Force clear everything and try again
        import gc
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        gc.collect()
        
        print("🔄 Attempting model load with nuclear fixes...")
        
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True,
            token=HF_TOKEN,
            low_cpu_mem_usage=True
        )
        
        print("🎉🎉🎉 NUCLEAR SUCCESS! MODEL LOADED! 🎉🎉🎉")
        print(f"Model device: {model.device}")
        print(f"Model dtype: {model.dtype}")
        print("Ready for Bengali OCR testing! 🚀")
        
    except Exception as e:
        print(f"❌ Even nuclear option failed: {e}")
        print(f"Error type: {type(e).__name__}")
        
        # Last desperate attempt: try to see what transformers is actually looking for
        print("\n🔍 Debugging what transformers is actually trying to import...")
        try:
            import transformers.dynamic_module_utils as dmu
            print("Available sys.modules with 'dots':")
            for key in sys.modules:
                if 'dots' in key.lower():
                    print(f"  - {key}: {sys.modules[key]}")
                    
            print("\nAvailable sys.modules with 'transformers_modules':")
            for key in sys.modules:
                if 'transformers_modules' in key:
                    print(f"  - {key}: {sys.modules[key]}")
                    
        except Exception as debug_e:
            print(f"Debug failed: {debug_e}")
        
        # Final attempt with manual class injection
        print("\n🩹 Final attempt: Manual class injection...")
        try:
            # Get the actual class from our loaded module
            ocr_module = sys.modules.get("modeling_dots_ocr")
            if ocr_module and hasattr(ocr_module, 'DotsOCRForConditionalGeneration'):
                DotsOCRClass = ocr_module.DotsOCRForConditionalGeneration
                
                # Manually inject into transformers
                from transformers import AutoModelForCausalLM
                AutoModelForCausalLM._model_mapping._extra_content["DotsOCRForConditionalGeneration"] = DotsOCRClass
                
                print("✅ Manually injected model class")
                
                # Try loading with the injected class
                model = DotsOCRClass.from_pretrained(
                    model_id,
                    torch_dtype=torch.bfloat16,
                    device_map="auto",
                    token=HF_TOKEN
                )
                
                print("🎉 SUCCESS WITH MANUAL INJECTION!")
                print(f"Model: {type(model)}")
                
            else:
                print("❌ Could not find DotsOCRForConditionalGeneration class")
                
        except Exception as final_e:
            print(f"❌ Manual injection failed: {final_e}")
            print("\n💀 All methods exhausted. This model has severe import issues.")
            print("🔄 RESTART KERNEL and try a different approach or different model.")

else:
    print("❌ Nuclear fix failed to load modules")

💥 NUCLEAR OPTION ACTIVATED
🚀 Nuclear option: Direct import system patching...
✅ All required files found
📦 Manually loading modules...
✅ Loaded configuration_dots
❌ Module loading failed: attempted relative import with no known parent package
❌ Nuclear fix failed to load modules


Traceback (most recent call last):
  File "/tmp/ipykernel_224/979877581.py", line 42, in nuclear_fix_dots_ocr
    spec.loader.exec_module(vision_module)
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f/modeling_dots_vision.py", line 10, in <module>
    from .configuration_dots import DotsVisionConfig
ImportError: attempted relative import with no known parent package


In [12]:
# ULTIMATE FIX: Patch file contents before loading
import sys
import importlib.util
from pathlib import Path
import re

def ultimate_fix_dots_ocr():
    """
    The ultimate fix: patch all file contents to remove relative imports
    """
    print("🔥 ULTIMATE FIX: Patching all file contents...")
    
    # Get file paths
    snapshot_dir = Path("/root/.cache/huggingface/hub/models--rednote-hilab--dots.ocr/snapshots/42f41787f2475c0894d75a9e30e49cc1a6df519f")
    
    files = {
        "config": snapshot_dir / "configuration_dots.py",
        "vision": snapshot_dir / "modeling_dots_vision.py", 
        "ocr": snapshot_dir / "modeling_dots_ocr.py"
    }
    
    # Check all files exist
    for name, path in files.items():
        if not path.exists():
            print(f"❌ {name} file not found: {path}")
            return False
    
    print("✅ All files found")
    
    # STEP 1: Read and patch all file contents
    patched_contents = {}
    
    try:
        # Patch configuration_dots.py
        config_content = files["config"].read_text()
        # Remove any relative imports (shouldn't have any, but just in case)
        patched_contents["config"] = config_content
        print("✅ Configuration file ready")
        
        # Patch modeling_dots_vision.py - this is likely where the relative import error comes from
        vision_content = files["vision"].read_text()
        
        # Replace any relative imports with absolute ones or remove them
        vision_content = re.sub(r'from\s+\.[\w.]*\s+import', 'import', vision_content)
        vision_content = re.sub(r'from\s+\.', 'from ', vision_content)
        
        # Add necessary imports at the top
        vision_imports = """
import torch
import torch.nn as nn
from transformers import PreTrainedModel
from transformers.modeling_outputs import BaseModelOutput
from transformers.utils import logging
import math
from typing import Optional, Tuple, Union
"""
        
        if "import torch" not in vision_content:
            vision_content = vision_imports + "\n" + vision_content
        
        patched_contents["vision"] = vision_content
        print("✅ Vision file patched")
        
        # Patch modeling_dots_ocr.py
        ocr_content = files["ocr"].read_text()
        
        # Replace relative imports with direct references
        ocr_content = re.sub(r'from\s+\.configuration_dots\s+import\s+(.*)', 
                           r'# Patched import - will be injected\n# \1 will be available directly', 
                           ocr_content)
        
        ocr_content = re.sub(r'from\s+\.modeling_dots_vision\s+import\s+(.*)',
                           r'# Patched import - will be injected\n# \1 will be available directly',
                           ocr_content)
        
        # Add necessary imports
        ocr_imports = """
import sys
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers.utils import logging
from typing import Optional, Tuple, Union, List
import math
"""
        
        if "import torch" not in ocr_content:
            ocr_content = ocr_imports + "\n" + ocr_content
            
        patched_contents["ocr"] = ocr_content
        print("✅ OCR file patched")
        
    except Exception as e:
        print(f"❌ File patching failed: {e}")
        return False
    
    # STEP 2: Load modules with patched content
    try:
        print("📦 Loading patched modules...")
        
        # Load config module
        config_spec = importlib.util.spec_from_loader("configuration_dots", loader=None)
        config_module = importlib.util.module_from_spec(config_spec)
        exec(patched_contents["config"], config_module.__dict__)
        sys.modules["configuration_dots"] = config_module
        print("✅ Config module loaded")
        
        # Load vision module
        vision_spec = importlib.util.spec_from_loader("modeling_dots_vision", loader=None)
        vision_module = importlib.util.module_from_spec(vision_spec)
        exec(patched_contents["vision"], vision_module.__dict__)
        sys.modules["modeling_dots_vision"] = vision_module
        print("✅ Vision module loaded")
        
        # Inject dependencies into OCR module namespace before loading
        ocr_namespace = {
            '__name__': 'modeling_dots_ocr',
            '__file__': str(files["ocr"]),
            'DotsVisionTransformer': getattr(vision_module, 'DotsVisionTransformer', None),
            'DotsVisionConfig': getattr(config_module, 'DotsVisionConfig', None),
            'DotsOCRConfig': getattr(config_module, 'DotsOCRConfig', None),
        }
        
        # Add any missing classes that might be needed
        for attr_name in dir(config_module):
            if not attr_name.startswith('_'):
                ocr_namespace[attr_name] = getattr(config_module, attr_name)
                
        for attr_name in dir(vision_module):
            if not attr_name.startswith('_'):
                ocr_namespace[attr_name] = getattr(vision_module, attr_name)
        
        # Load OCR module with injected dependencies
        ocr_spec = importlib.util.spec_from_loader("modeling_dots_ocr", loader=None)
        ocr_module = importlib.util.module_from_spec(ocr_spec)
        
        # Execute with pre-loaded dependencies
        exec(patched_contents["ocr"], ocr_namespace)
        
        # Copy everything back to the module
        for key, value in ocr_namespace.items():
            setattr(ocr_module, key, value)
        
        sys.modules["modeling_dots_ocr"] = ocr_module
        print("✅ OCR module loaded")
        
        # Register in transformers_modules namespace
        sys.modules["transformers_modules.rednote-hilab.dots.configuration_dots"] = config_module
        sys.modules["transformers_modules.rednote-hilab.dots.modeling_dots_vision"] = vision_module  
        sys.modules["transformers_modules.rednote-hilab.dots.modeling_dots_ocr"] = ocr_module
        sys.modules["transformers_modules.rednote-hilab.dots"] = ocr_module
        
        # Create package hierarchy
        class FakePackage:
            def __init__(self, name):
                self.__name__ = name
                self.__path__ = []
                
        sys.modules["transformers_modules"] = FakePackage("transformers_modules")
        sys.modules["transformers_modules.rednote-hilab"] = FakePackage("transformers_modules.rednote-hilab")
        
        print("✅ All modules registered in transformers namespace")
        
    except Exception as e:
        print(f"❌ Module loading failed: {e}")
        import traceback
        traceback.print_exc()
        return False
    
    return True

# Execute ultimate fix
print("🔥🔥🔥 ULTIMATE FIX ACTIVATED 🔥🔥🔥")
print("=" * 60)

if ultimate_fix_dots_ocr():
    print("\n✅ ULTIMATE FIX COMPLETED!")
    print("🚀 Attempting model load...")
    
    try:
        # Clear cache
        import gc
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
        # Try loading
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto", 
            trust_remote_code=True,
            token=HF_TOKEN,
            low_cpu_mem_usage=True
        )
        
        print("🎉🎉🎉 ULTIMATE SUCCESS! 🎉🎉🎉")
        print(f"Model loaded: {type(model)}")
        print(f"Model device: {model.device}")
        print(f"Model dtype: {model.dtype}")
        print("🚀 READY FOR BENGALI OCR! 🚀")
        
    except Exception as e:
        print(f"❌ Model loading still failed: {e}")
        
        # Check what we actually loaded
        print("\n🔍 Checking loaded modules...")
        ocr_module = sys.modules.get("modeling_dots_ocr")
        if ocr_module:
            print("Available classes in OCR module:")
            for attr in dir(ocr_module):
                if not attr.startswith('_') and 'OCR' in attr:
                    print(f"  - {attr}: {getattr(ocr_module, attr)}")
                    
            # Try direct instantiation
            if hasattr(ocr_module, 'DotsOCRForConditionalGeneration'):
                print("\n🎯 Trying direct class instantiation...")
                try:
                    ModelClass = ocr_module.DotsOCRForConditionalGeneration
                    model = ModelClass.from_pretrained(
                        model_id,
                        torch_dtype=torch.bfloat16,
                        device_map="auto",
                        token=HF_TOKEN
                    )
                    print("🎉 DIRECT INSTANTIATION SUCCESS!")
                    print(f"Model: {type(model)}")
                except Exception as direct_e:
                    print(f"❌ Direct instantiation failed: {direct_e}")
        else:
            print("❌ OCR module not found in sys.modules")
            
        print("\n💀 If this still fails, the model is fundamentally broken.")
        print("🔄 Consider using a different OCR model like microsoft/trocr-base-printed")

else:
    print("❌ Ultimate fix failed")
    print("💀 This model appears to have unfixable import issues")
    print("🔄 Recommend trying a different model")

🔥🔥🔥 ULTIMATE FIX ACTIVATED 🔥🔥🔥
🔥 ULTIMATE FIX: Patching all file contents...
✅ All files found
✅ Configuration file ready
✅ Vision file patched
✅ OCR file patched
📦 Loading patched modules...
✅ Config module loaded
❌ Module loading failed: No module named 'DotsVisionConfig'
❌ Ultimate fix failed
💀 This model appears to have unfixable import issues
🔄 Recommend trying a different model


Traceback (most recent call last):
  File "/tmp/ipykernel_224/668948942.py", line 112, in ultimate_fix_dots_ocr
    exec(patched_contents["vision"], vision_module.__dict__)
  File "<string>", line 10, in <module>
ModuleNotFoundError: No module named 'DotsVisionConfig'
